In [40]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [41]:
dataset_path = r"\\fauad.fau.de\shares\ANKI\Projects\Swallowing\Data\from_Melda"

In [42]:
# Overview spreadsheet of dataset
# we skip crf for now
#TODI; add Visit Info


def analyze_patient_data(base_directory):
    data = []
    error_data = []

    folders = os.listdir(base_directory)
    folders = [f for f in folders if f.isnumeric()]
    count = 0
    for patient_id in tqdm(folders):
        count += 1
        #if count > 5: break
        patient_folder = os.path.join(base_directory, patient_id)
        if not os.path.isdir(patient_folder):
            continue

        video_data = []
        csv_data = []

        # Step 1: Process video files
        for file in os.listdir(patient_folder):
            file_path = os.path.join(patient_folder, file)

            if file.endswith(".mp4"):# and "CRF8" not in file:
                full_video_id = file.split("_")[0]
                is_crf = "crf" in file.lower()
                video = cv2.VideoCapture(file_path)
                frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = video.get(cv2.CAP_PROP_FPS)
                duration = frame_count / fps if fps > 0 else 0
                video.release()

                #get video type
                if ("_fixed_" in file and is_crf is False) or ("_fixed_CRF8_" in file): video_type = "swallow"
                else: video_type = "full"


                extracted_swallow_id = None
                if video_type == "swallow":
                    extracted_swallow_id = int(file.split("_")[-1].split(".")[0])

                video_data.append({
                    "PatientID": patient_id,
                    "FullVideoID": full_video_id,
                    "ParsedFullVideoID": file,
                    "ExtractedSwallowID": extracted_swallow_id,
                    "VideoType": video_type,
                    "IsCRF": is_crf,
                    "FrameCount": frame_count,
                    "FPS": round(fps, 2),
                    "Duration": round(duration, 2),
                })

        # Step 2: Process CSV/Excel files
        for file in os.listdir(patient_folder):
            file_path = os.path.join(patient_folder, file)

            if (file.endswith(".xlsx") or file.endswith(".csv")) and "$" not in file:
                try:
                    if file.endswith(".xlsx"):
                        visit_data = pd.read_excel(file_path, engine='openpyxl')
                    else:
                        visit_data = pd.read_csv(file_path)

                    if "From frame" in visit_data.columns and "To frame" in visit_data.columns:
                        for idx, row in visit_data.iterrows():
                            swallow_id = idx  # Assume the row index matches the extracted swallow ID
                            matched_video = next((
                                v for v in video_data
                                if v["VideoType"] == "swallow" and v["ExtractedSwallowID"] == swallow_id
                            ), None)

                            if matched_video:
                                matched_video.update({
                                    "csv_FromFrame": row.get("From frame", None),
                                    "csv_ToFrame": row.get("To frame", None),
                                    "csv_Paradigm": row.get("Paradigm", None),
                                    "csv_EventDescription": row.get("Event description", None),
                                    "csv_Comment": row.get("Comment", None),
                                })
                            else:
                                #print(f"No matching video found for swallow ID {swallow_id} in file {file}.")
                                error_data.append({
                                    "PatientID": patient_id,
                                    "SwallowID": swallow_id,
                                    "File": file,
                                })

                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

        # Add all video data for the patient to the overall data
        data.extend(video_data)

    # Create DataFrame from the collected data
    df = pd.DataFrame(data)
    error_df = pd.DataFrame(error_data)
    return df, error_df

# Example usage
df_videos, df_errors = analyze_patient_data(dataset_path)


100%|██████████| 483/483 [13:48<00:00,  1.72s/it]


In [43]:
df_videos

,PatientID,FullVideoID,ParsedFullVideoID,ExtractedSwallowID,VideoType,IsCRF,FrameCount,FPS,Duration,csv_FromFrame,csv_ToFrame,csv_Paradigm,csv_EventDescription,csv_Comment
0,10000,Pt10000,Pt10000_Visit1_fixed.mp4,NaN,full,False,5493,29.92,183.61,NaN,NaN,NaN,NaN,NaN
1,102,Pt102,Pt102_Visit1_fixed.mp4,NaN,full,False,5070,29.92,169.47,NaN,NaN,NaN,NaN,NaN
2,102,Pt102,Pt102_Visit1_fixed_000.mp4,0.0,swallow,False,55,30.00,1.83,193.0,264.0,1 mL liquid bolus,Swallow,NaN
3,102,Pt102,Pt102_Visit1_fixed_001.mp4,1.0,swallow,False,48,30.00,1.60,765.0,845.0,1 mL liquid bolus,Swallow,NaN
4,102,Pt102,Pt102_Visit1_fixed_002.mp4,2.0,swallow,False,36,30.00,1.20,1059.0,1101.0,1 mL liquid bolus,Swallow,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10952,996,Pt996,Pt996_Visit1_fixed_022.mp4,22.0,swallow,False,53,30.00,1.77,7444.0,7496.0,1 mL liquid bolus,Swallow,NaN
10953,9972,Pt9972,Pt9972_Visit1_fixed.mp4,NaN,full,False,4593,30.00,153.10,NaN,NaN,NaN,NaN,NaN
10954,9972,Pt9972,Pt9972_Visit2_fixed.mp4,NaN,full,False,4083,29.92,136.48,NaN,NaN,NaN,NaN,NaN
10955,9972,Pt9972,Pt9972_Visit3_fixed.mp4,NaN,full,False,30810,29.97,1028.03,NaN,NaN,NaN,NaN,NaN


In [39]:
df_videos.to_csv("MeldaDatasetAnnotations.csv", index=False)

In [44]:
def summarize_patient_videos(df_videos):
    # Group by PatientID and calculate summary statistics
    summary = df_videos.groupby("PatientID").apply(
        lambda group: pd.Series({
            "NumberOfFullVideos": (group["VideoType"] == "full").sum(),
            "NumberOfSwallowVideos": (group["VideoType"] == "swallow").sum()
        })
    ).reset_index()

    return summary
df_patient_summary = summarize_patient_videos(df_videos)

C:\Users\ly37xoco\AppData\Local\Temp\ipykernel_19720\2428816036.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = df_videos.groupby("PatientID").apply(


In [45]:
df_patient_summary

,PatientID,NumberOfFullVideos,NumberOfSwallowVideos
0,10000,1,0
1,102,6,94
2,1020,3,68
3,1023,8,57
4,1053,3,49
...,...,...,...
478,9943,2,0
479,9957,1,0
480,996,1,23
481,9972,3,0


In [46]:
pd.DataFrame(df_errors)

,PatientID,SwallowID,File
0,164,16,Pt164_Visit1_fixed.xlsx
1,164,17,Pt164_Visit1_fixed.xlsx
2,164,18,Pt164_Visit1_fixed.xlsx
3,164,19,Pt164_Visit1_fixed.xlsx
4,164,20,Pt164_Visit1_fixed.xlsx
5,164,21,Pt164_Visit1_fixed.xlsx
6,164,22,Pt164_Visit1_fixed.xlsx
7,164,23,Pt164_Visit1_fixed.xlsx
8,4446,13,Pt4446_Visit1_fixed.xlsx
9,4446,14,Pt4446_Visit1_fixed.xlsx


In [28]:
df_videos.to_csv("MeldaDatasetAnnotations.csv", index=True)

In [47]:
# check how many patient ids have no video type swallow in the df
processed_patients = df_patient_summary[df_patient_summary["NumberOfSwallowVideos"] > 0]["PatientID"].count()
unprocessed_patients = df_patient_summary[df_patient_summary["NumberOfSwallowVideos"] == 0]["PatientID"].count()
print(f"Processed Patients: {processed_patients}, Unprocessed Patients: {unprocessed_patients}, Total Patients: {df_patient_summary['PatientID'].count()}, Percentage: {round(processed_patients / df_patient_summary['PatientID'].count() * 100, 2)}%")

Processed Patients: 277, Unprocessed Patients: 206, Total Patients: 483, Percentage: 57.35%


In [48]:
# how many swallow videos do we have in total
df_patient_summary["NumberOfSwallowVideos"].sum()

np.int64(9824)

In [3]:
def analyze_patient_data(base_directory):
    data = []

    folders = os.listdir(base_directory)
    # only check folders in base directory, also only with numbers
    folders = ([f for f in folders if f.isnumeric()])
    # Iterate over each patient folder
    for patient_id in tqdm(folders):
        patient_folder = os.path.join(base_directory, patient_id)
        if not os.path.isdir(patient_folder):
            continue  # Skip if not a folder

        # Initialize patient data
        patient_data = {
            "PatientID": patient_id,
            "VisitCount": 0,
            "ExcelRowCounts": [],  # List of row counts per video
            "FullVideosTotalFrames": [],  # Frame counts for full videos
            "FullVideosDurations": [],  # Durations for full videos (in seconds)
            "FullVideosFPS": [],  # FPS for full videos
            "FullVideosNames": [],  # Names of full videos
            "SwallowFrames": [],  # Frame counts for swallow videos
            "SwallowDurations": [],  # Durations for swallow videos (in seconds)
            "SwallowFPS": [],  # FPS for swallow videos
            "NegativeFramesVideos": "",  # Video IDs with negative frames
        }

        # Iterate over each file in the patient folder
        for file in os.listdir(patient_folder):
            file_path = os.path.join(patient_folder, file)

            # Analyze full videos
            if file.endswith(".mp4"):
                video = cv2.VideoCapture(file_path)
                frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = video.get(cv2.CAP_PROP_FPS)
                duration = frame_count / fps if fps > 0 else 0  # Avoid division by zero
                video.release()

                # ignore crf8
                if "crf8" in file.lower():
                    continue
                elif "_fixed_" in file:  # Swallow videos
                    patient_data["SwallowFrames"].append(frame_count)
                    patient_data["SwallowDurations"].append(duration)
                    patient_data["SwallowFPS"].append(fps)
                elif "_fixed" in file:  # Full visit videos
                    patient_data["VisitCount"] += 1
                    patient_data["FullVideosTotalFrames"].append(frame_count)
                    patient_data["FullVideosDurations"].append(duration)
                    patient_data["FullVideosFPS"].append(fps)
                    patient_data["FullVideosNames"].append(file)

            # Analyze Excel/CSV files
            if (file.endswith(".xlsx") or file.endswith(".csv")) and not "$" in file:  # filter out "hidden" ~$ tmp files
                try:
                    # Read file
                    if file.endswith(".xlsx"):
                        visit_data = pd.read_excel(file_path, engine='openpyxl')
                    else:
                        visit_data = pd.read_csv(file_path)

                    # Count rows and append to ExcelRowCounts
                    row_count = len(visit_data)
                    patient_data["ExcelRowCounts"].append(row_count)

                    # Check for negative frames
                    if "From frame" in visit_data.columns and "To frame" in visit_data.columns:
                        frame_durations = visit_data["To frame"] - visit_data["From frame"]
                        negative_frames_videos = visit_data.loc[frame_durations < 0]
                        if not negative_frames_videos.empty:
                            if patient_data["NegativeFramesVideos"]:
                                patient_data["NegativeFramesVideos"] += f", {file}"
                            else:
                                patient_data["NegativeFramesVideos"] = file
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")
                    continue  # Skip problematic files

        # Add to the overall data
        data.append(patient_data)


    # Create DataFrame from the collected data
    df = pd.DataFrame(data)

    return df

df_folders = analyze_patient_data(dataset_path)


100%|██████████| 483/483 [12:39<00:00,  1.57s/it]


In [4]:
df_folders

,PatientID,VisitCount,ExcelRowCounts,FullVideosTotalFrames,FullVideosDurations,FullVideosFPS,FullVideosNames,SwallowFrames,SwallowDurations,SwallowFPS,NegativeFramesVideos
0,10000,1,[],[5493],[183.6100278551532],[29.916666666666668],[Pt10000_Visit1_fixed.mp4],[],[],[],
1,102,3,"[23, 42, 29]","[5070, 11822, 12866]","[169.47075208913648, 395.1643454038997, 430.06...","[29.916666666666668, 29.916666666666668, 29.91...","[Pt102_Visit1_fixed.mp4, Pt102_Visit2_fixed.mp...","[55, 48, 36, 44, 32, 53, 39, 87, 40, 81, 57, 1...","[1.8333333333333333, 1.6, 1.2, 1.4666666666666...","[30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30....",
2,1020,3,"[35, 33]","[7229, 8533, 10062]","[241.63788300835654, 284.71776666666665, 336.3...","[29.916666666666668, 29.97002997002997, 29.916...","[Pt1020_Visit1_fixed.mp4, Pt1020_Visit2_fixed....","[38, 33, 43, 35, 33, 44, 44, 55, 37, 37, 46, 3...","[1.2666666666666666, 1.1, 1.4333333333333333, ...","[30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30....",
3,1023,8,"[4, 3, 4, 1, 3, 11, 31]","[811, 871, 661, 391, 841, 631, 9842, 6989]","[27.108635097493035, 29.114206128133702, 22.09...","[29.916666666666668, 29.916666666666668, 29.91...","[Pt1023_Visit1_1_fixed.mp4, Pt1023_Visit1_2_fi...","[170, 41, 37, 41, 242, 53, 44, 164, 50, 38, 40...","[5.666666666666667, 1.3666666666666667, 1.2333...","[30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30....",
4,1053,3,"[3, 19, 27]","[1998, 7863, 8072]","[66.6, 262.8300835654596, 269.81615598885793]","[30.0, 29.916666666666668, 29.916666666666668]","[Pt1053_Visit1_fixed.mp4, Pt1053_Visit2_fixed....","[62, 50, 109, 101, 53, 118, 130, 59, 76, 61, 1...","[2.066666666666667, 1.6666666666666667, 3.6333...","[30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30....",
...,...,...,...,...,...,...,...,...,...,...,...
478,9943,2,[],"[7969, 7863]","[265.6333333333333, 262.8300835654596]","[30.0, 29.916666666666668]","[Pt9943_Visit1_fixed.mp4, Pt9943_Visit2_fixed....",[],[],[],
479,9957,1,[],[9481],[316.9136490250696],[29.916666666666668],[Pt9957_Visit1_fixed.mp4],[],[],[],
480,996,1,[23],[7682],[256.77994428969356],[29.916666666666668],[Pt996_Visit1_fixed.mp4],"[76, 119, 30, 47, 44, 62, 36, 51, 56, 32, 42, ...","[2.533333333333333, 3.966666666666667, 1.0, 1....","[30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30....",
481,9972,3,[],"[4593, 4083, 30810]","[153.1, 136.47910863509748, 1028.027]","[30.0, 29.916666666666668, 29.97002997002997]","[Pt9972_Visit1_fixed.mp4, Pt9972_Visit2_fixed....",[],[],[],


In [ ]:
'''
def analyze_patient_data(base_directory):
    data = []
    folders = os.listdir(base_directory)
    # only check folders in base directory, also only with numbers
    folders = ([f for f in folders if f.isnumeric()])
    # Iterate over each patient folder
    for patient_id in tqdm(folders):
        #print(patient_id)
        patient_folder = os.path.join(base_directory, patient_id)
        if not os.path.isdir(patient_folder):
            continue  # Skip if not a folder

        # Initialize patient data
        patient_data = {
            "PatientID": patient_id,
            "VisitCount": 0,
            "IsProcessed": False,
            "ProcessedSwallows": 0,
            "FramesPerSwallow": [],
            "AverageFramesPerSwallow": None,
            "Paradigms": [],
            "IsCRF8": False,
        }

        # Iterate over each file in the patient folder
        for file in os.listdir(patient_folder):
            if file.endswith(".mp4"):
                # Count visits based on video filenames
                if "_Visit" in file:
                    patient_data["VisitCount"] += 1
                    if "CRF8" in file:
                        patient_data["IsCRF8"] = True

            if (file.endswith(".xlsx") or file.endswith(".csv")) and not "$" in file: # filter out "hidden" ~$ tmp files
                # Identify processed data
                if "_Visit" in file:
                    patient_data["IsProcessed"] = True

                    # Read the file (CSV or Excel) to extract swallow information
                    file_path = os.path.join(patient_folder, file)
                    if file.endswith(".xlsx"):
                        visit_data = pd.read_excel(file_path,  engine='openpyxl')
                    else:
                        visit_data = pd.read_csv(file_path)

                    # Extract number of swallows and paradigms
                    patient_data["ProcessedSwallows"] += len(visit_data)
                    if "From frame" in visit_data and "To frame" in visit_data:
                        frames = visit_data["To frame"] - visit_data["From frame"]
                        # check for negative values
                        if (frames < 0).sum() > 0:
                            pass #print(f"Negative frame values in {file_path} of patient {patient_id}")
                        patient_data["FramesPerSwallow"].extend(frames)
                        patient_data["AverageFramesPerSwallow"] = frames.mean()
                    if "Paradigm" in visit_data:
                        patient_data["Paradigms"].extend(visit_data["Paradigm"].unique())

        # Add to the overall data
        data.append(patient_data)
        #break
    # Create DataFrame from the collected data
    df = pd.DataFrame(data)

    # Clean up Paradigms column to be unique lists
    df["Paradigms"] = df["Paradigms"].apply(lambda x: list(set(x)) if isinstance(x, list) else [])

    return df

# Directory containing the patient data
df_folders_excel = analyze_patient_data(dataset_path)
'''

In [ ]:
df_folders

In [ ]:
# create column num of swallow frames
df_folders["NumSwallowFrames"] = df_folders["SwallowFrames"].apply(lambda x: len(x))
# create column num of total excel swallows (sum of all swallows in all excel files)
df_folders["NumTotalExcelSwallows"] = df_folders["ExcelRowCounts"].apply(lambda x: sum(x))
df_folders

How much patient data already processed?

In [ ]:
df_folders["IsProcessed"] = df_folders["SwallowFrames"].apply(lambda x: len(x) > 0)
df_folders_processed = df_folders[df_folders["IsProcessed"]]
df_folders["IsProcessed"].value_counts()

Videos per patient distribution

In [ ]:
df_folders["VisitCount"].value_counts()


In [ ]:
# Plot the distribution of visits per patient
plt.figure(figsize=(10, 6))
plt.hist(df_folders["VisitCount"], bins=range(1, df_folders["VisitCount"].max() + 2), edgecolor='black')
# add x amd y labels
plt.xlabel("Number of Visits")
plt.ylabel("Number of Patients")
plt.title("Distribution of Visits per Patient")
# save plot
plt.savefig("visits_per_patient.png")

for the processed data: ...

how many swallows per patient?

In [ ]:
# Plot the distribution of processed swallows per patient
plt.figure(figsize=(10, 6))
plt.hist(df_folders_processed["NumSwallowFrames"], bins=range(1, df_folders_processed["NumSwallowFrames"].max(), 10), edgecolor='black')
# add x amd y labels
plt.xlabel("Number of Swallows")
plt.ylabel("Number of Patients")
plt.title("Distribution of Swallows per Patient")
# save plot
plt.savefig("swallows_per_patient.png")

how many frames per swallow?

In [ ]:
df_folders_processed

In [ ]:
# need to explode frames list col
df_frames = df_folders_processed.explode("SwallowFrames")
df_frames["SwallowFrames"] = df_frames["SwallowFrames"].astype(int)
df_frames["SwallowFrames"].describe()

In [ ]:
# filter out values above 150
df_frames[df_frames["SwallowFrames"] > 500]

In [ ]:
# plot distribution of frames per swallow
plt.figure(figsize=(10, 6))
plt.hist(df_frames["SwallowFrames"], bins=range(1, 400, 20), edgecolor='black') #df_frames["SwallowFrames"].max() + 2), edgecolor='black')
plt.xlabel("Number of Frames")
plt.ylabel("Number of Swallows")
plt.title("Distribution of Frames per Swallow")
plt.savefig("frames_per_swallow.png")

 noch sekunden pro video + fps berechnen

In [ ]:
df_folders

In [ ]:
# get fullvideosfps and round to full number and show value counts
# do explode first
df_fps = df_folders.explode("FullVideosFPS")
df_fps["FullVideosFPS"] = df_fps["FullVideosFPS"].round(0).astype(int)
df_fps["FullVideosFPS"].value_counts()


In [ ]:
df_folders

numswallowframes != numtotalexcelswallows

In [ ]:
# how many full videos do we have in total
df_folders["NumSwallowFrames"].sum()

In [ ]:
# sum all visitcount values
totalcount = df_folders["VisitCount"].sum()
totalcount

In [ ]:
# how many swallow videos to we have in total (number of swallow frames)
df_folders["NumSwallowFrames"].sum()

In [ ]:
# plot distribution of frames per swallow
# explode SwallowDurations
df_swallow_durations = df_folders.explode("SwallowDurations")
# remove nan values
df_swallow_durations = df_swallow_durations.dropna(subset=["SwallowDurations"])
plt.figure(figsize=(10, 6))
plt.hist(df_swallow_durations["SwallowDurations"], bins=range(0, 25), edgecolor='black')
plt.xlabel("Number of seconds")
plt.ylabel("Number of Swallow videos")
plt.title("Distribution of Seconds per Swallow video")
plt.savefig("frames_per_swallow.png")

In [ ]:
df_swallow_durations["SwallowDurations"]#.head()#.describe()

### Analyze demographics

In [ ]:
# Load data
df_patient_info = pd.read_csv('Patho_VFSS_Info_1205.csv')
df_patient_info.tail()

In [ ]:
# column explanations
df_patient_info[df_patient_info["UID"].isna()]
# drop explanation rows
df_patient_info = df_patient_info.dropna(subset=["UID"])

Check NaN

In [ ]:
# which columns have nan values?
df_patient_info.isna().sum()

In [ ]:
df_patient_info.head()

preprocess gender

In [ ]:
df_patient_info["Gender"] = df_patient_info["Gender"].replace({"0": "M", "1": "F"})

preprocess age

In [ ]:
# process columns with multiple values per cell
# Step 1: Convert the Age column to lists
def process_age(value):
    if pd.isna(value):  # Check for NaN
        return None  # Return an empty list for NaN
    elif isinstance(value, str):
        # Split by any combination of commas and spaces, then convert to integers
        return [int(x.strip()) for x in value.replace(',', ', ').split(', ') if x.strip().isdigit()]
    else:
        return [int(value)]

df_patient_info['Age'] = df_patient_info['Age'].apply(process_age)

# Step 2: Expand rows so each age value gets its own row
df_patient_age = df_patient_info.explode('Age')
# drop nan values
df_patient_age = df_patient_age.dropna(subset=["Age"])

# Step 3: Create an ordering index
df_patient_age['Index'] = df_patient_age.groupby('UID').cumcount() + 1

# Step 4: Convert the Age column to integers
#df_patient_age = df_patient_age[['UID','Age','Year of Birth']].copy()
# convert col to int
df_patient_age['Age'] = df_patient_age['Age'].astype(int)
df_patient_age

### patient info

In [ ]:
# how many patients?
len(df_patient_info["UID"].unique())

age

In [ ]:
# plot distribution of age
print("Age Stats:")
print(df_patient_age["Age"].describe())
# na values are age = 0
print(f"\nPatients with missing age: {df_patient_info["Age"].isna().sum()}")

In [ ]:
# Plot the age distribution

# Determine the bin range
min_age = df_patient_age["Age"].min()
max_age = df_patient_age["Age"].max()
bins = range(min_age, max_age + 2, 5)  # Bin width of 2 years

# Plot histogram for all data combined (no gender differentiation)
plt.figure(figsize=(10, 6))
plt.hist(
    df_patient_age["Age"],
    bins=bins,
    density=True,
    edgecolor='black'
)
plt.ylim(0, 0.01)  # Adjust based on expected density range
plt.yticks([0, 0.01, 0.02, 0.03, 0.04, 0.05])  # Standardized ticks

# Add titles and labels
plt.title("Normalized Age Distribution")
plt.xlabel("Age")
plt.ylabel("Density")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

gender

In [ ]:
# show gender stats (0 = Male, 1 = Female)
print(f"Missing Gender: {df_patient_info["Gender"].isna().sum()}")
print(df_patient_info["Gender"].value_counts())
print(round(df_patient_info["Gender"].value_counts() / len(df_patient_info["Gender"].dropna()), 2))

In [ ]:
# make a plot showing the gender distribution
plt.figure(figsize=(6, 6))


age distribution per gender


In [ ]:
# Adjust bins to be consistent for both genders
plt.figure(figsize=(10, 6))

# Determine a consistent bin range for both genders
min_age = df_patient_age["Age"].min()
max_age = df_patient_age["Age"].max()
bins = range(min_age, max_age + 2, 5)  # Bin width of 2 years

for gender in df_patient_age["Gender"].unique():
    subset = df_patient_age[df_patient_age["Gender"] == gender]
    plt.hist(subset["Age"], bins=bins, alpha=0.6, density=True, label=f"{gender}", edgecolor='black')

plt.ylim(0, 0.01)  # Adjust based on expected density range
plt.yticks([0, 0.01, 0.02, 0.03, 0.04, 0.05])  # Standardized ticks

plt.title("Normalized Age Distribution by Gender with Consistent Bins")
plt.xlabel("Age")
plt.ylabel("Density")
plt.legend(title="Gender")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# different type of visualization
# Preparing data for the demographic plot
bin_width = 5
min_age = 15#df_patient_age["Age"].min()
max_age = df_patient_age["Age"].max()
bins = range(min_age, max_age + bin_width, bin_width)

# Count the number of patients in each age bin for each gender
age_distribution = (
    df_patient_age.groupby(["Gender", pd.cut(df_patient_age["Age"], bins)])
    .size()
    .unstack(level=0, fill_value=0)
)

# Make male counts negative for left side of the plot
age_distribution["M"] = -age_distribution["M"]

# Plotting the demographic graphic
plt.figure(figsize=(12, 8))
age_bins_labels = [f"{int(interval.left)}-{int(interval.right)}" for interval in age_distribution.index]

plt.barh(age_bins_labels, age_distribution["M"], color="blue", alpha=0.7, label="Male")
plt.barh(age_bins_labels, age_distribution["F"], color="pink", alpha=0.7, label="Female")

plt.title("Age Distribution by Gender")
plt.xlabel("Number of Patients")
plt.ylabel("Age Group")
plt.axvline(0, color='black', linewidth=0.8)
plt.legend(title="Gender")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
df_patient_info

 acquistion dates by adding age(s) to birth year

In [ ]:
# add to the df_patient_age column acquisition date
df_patient_age["Acquisition Date"] = (df_patient_age["Year of Birth"] + df_patient_age["Age"]).astype(int)
df_patient_age

In [ ]:
# plot distribution of acquisition dates per year
# there is incorrect value 74 for one row
# Count acquisitions per year
acquisitions_per_year = df_patient_age['Acquisition Date'].value_counts().reindex(range(2007, 2022), fill_value=0).sort_index()
# Plot the distribution
plt.figure(figsize=(10, 6))
acquisitions_per_year.plot(kind='bar', edgecolor='black')
plt.title('Number of Acquisitions Per Year (2007-2021)')
plt.xlabel('Year')
plt.ylabel('Number of Acquisitions')
plt.grid(axis='y', linestyle='', alpha=0.7)
plt.xticks(rotation=45)
plt.savefig("acquisitions_per_year.png")
plt.show()

In [ ]:
df_patient_info

types of medical diagnosis

In [ ]:
# Convert the string column into lists
def process_conditions(value):
    if pd.isna(value):  # Handle NaN values
        return []
    # Split on commas and strip extra whitespace, remove empty strings
    return [item.strip() for item in value.split(',') if item.strip()]

df_patient_info["Associated Medical diagnosis"] = df_patient_info["Associated Medical diagnosis"].apply(process_conditions)
df_patient_info["Associated Medical diagnosis"].head()

In [ ]:
#

count of conditions

In [ ]:
df_medical_conditions = df_patient_info.explode("Associated Medical diagnosis")
# before exploding, make all values lowercase
df_medical_conditions["Associated Medical diagnosis"] = df_medical_conditions["Associated Medical diagnosis"].str.lower()
#replace nan with empty string
df_medical_conditions["Associated Medical diagnosis"] = df_medical_conditions["Associated Medical diagnosis"].fillna("")
df_medical_conditions["Associated Medical diagnosis"].value_counts()
# show top 10 conditions with pretty dataframe
df_medical_conditions["Associated Medical diagnosis"].value_counts()#.head(10)

In [ ]:
# how many unique medical diagnosis?
print(f"Unique Medical Diagnosis: {df_medical_conditions['Associated Medical diagnosis'].explode().nunique()}")

# how many diagnosis conditions contain the word "swallowing" or "dysphagia"?
print(f"Unique Swallowing Conditions: {df_medical_conditions['Associated Medical diagnosis'].drop_duplicates().str.contains('swallowing|dysphagia', case=False, na=False).sum()}")

# how many patients have diagnosis that contains words "swallowing" or "dysphagia"?
print(f"Patients with swallowing diagnosis: {df_medical_conditions["Associated Medical diagnosis"].str.contains('swallowing|dysphagia', case=False, na=False).sum()} (out of {len(df_patient_info)})")
# create column for "has_dysphagia"
df_medical_conditions["HasDysphagia"] = df_medical_conditions["Associated Medical diagnosis"].str.contains('swallowing|dysphagia', case=False, na=False)


In [ ]:
df_medical_conditions["HasDysphagia"].value_counts()

In [ ]:
df_medical_conditions[df_medical_conditions["HasDysphagia"] == True]["Associated Medical diagnosis"].value_counts().to_csv("dysphagia_conditions.csv")

how many conditions per patient?

In [ ]:
# how many conditions per patient?
df_medical_conditions["UID"].value_counts().value_counts()

In [ ]:
# which diagnosis often happen together with dysphagia which ia not a text containing dysphagia or swallowing?
dysphagia_uids = df_medical_conditions[df_medical_conditions["HasDysphagia"] == True]["UID"].tolist()
df_dysphagia = df_medical_conditions[df_medical_conditions["UID"].isin(dysphagia_uids)]
# filter out rows where dysphagia is in the diagnosis
df_dysphagia_tmp = df_dysphagia[~df_dysphagia["Associated Medical diagnosis"].str.contains('swallowing|dysphagia', case=False, na=False)]
df_dysphagia_tmp["Associated Medical diagnosis"].value_counts()

In [ ]:
# how many disphagia patients have column aspiration pneumonia?
df_dysphagia[df_dysphagia["Associated Medical diagnosis"].str.contains("Has patient been diagnosed with aspiration pneumonia?", case=False, na=False)].shape[0]

In [ ]:
df_medical_conditions